In [21]:
import pymc3 as pm
import numpy as np
import pandas as pd
import theano.tensor as tt
from scipy.special import expit
invlogit = lambda x: 1/(1 + tt.exp(-x))

In [23]:
df = pd.read_csv('train.csv')
print(df.head())
y = np.asarray(df.target)
X = np.array(df.iloc[:, 2:302])
df2 = pd.read_csv('test.csv')
df2.head()
X2 = np.array(df2.iloc[:, 1:301])

   id  target      0      1      2      3      4      5      6      7  ...  \
0   0     1.0 -1.067 -1.114 -0.616  0.376  1.090  0.467 -0.422  0.460  ...   
1   1     0.0 -0.831  0.271  1.716  1.096  1.731 -0.197  1.904 -0.265  ...   
2   2     0.0  0.099  1.390 -0.732 -1.065  0.005 -0.081 -1.450  0.317  ...   
3   3     1.0 -0.989 -0.916 -1.343  0.145  0.543  0.636  1.127  0.189  ...   
4   4     0.0  0.811 -1.509  0.522 -0.360 -0.220 -0.959  0.334 -0.566  ...   

     290    291    292    293    294    295    296    297    298    299  
0  0.220 -0.339  0.254 -0.179  0.352  0.125  0.347  0.436  0.958 -0.824  
1 -0.765 -0.735 -1.158  2.554  0.856 -1.506  0.462 -0.029 -1.932 -0.343  
2 -1.311  0.799 -1.001  1.544  0.575 -0.309 -0.339 -0.148 -0.646  0.725  
3 -1.370  1.093  0.596 -0.589 -0.649 -0.163 -0.958 -1.081  0.805  3.401  
4 -0.178  0.718 -1.017  1.249 -0.596 -0.445  1.751  1.442 -0.393 -0.643  

[5 rows x 302 columns]


In [13]:
def get_model(y, X):
    model = pm.Model()
    with model:
        xi = pm.Bernoulli('xi', .05, shape=X.shape[1]) #inclusion probability for each variable
        alpha = pm.Normal('alpha', mu = 0, sd = 5) # Intercept
        beta = pm.Normal('beta', mu = 0, sd = .75 , shape=X.shape[1]) #Prior for the non-zero coefficients
        p = pm.math.dot(X, xi * beta) #Deterministic function to map the stochastics to the output
        y_obs = pm.Bernoulli('y_obs', invlogit(p + alpha),  observed=y)  #Data likelihood
    return model
model1 = get_model(y, X)
with model1:
    trace = pm.sample(2000, random_seed = 4816, cores = 1, progressbar = True, chains = 2)

Sequential sampling (2 chains in 1 job)
CompoundStep
>BinaryGibbsMetropolis: [xi]
>NUTS: [beta, alpha]
100%|██████████| 2500/2500 [02:23<00:00, 17.38it/s]


In [14]:
results = pd.DataFrame({'var': np.arange(300), 
                        'inclusion_probability':np.apply_along_axis(np.mean, 0, trace['xi']),
                       'beta':np.apply_along_axis(np.mean, 0, trace['beta']),
                       'beta_given_inclusion': np.apply_along_axis(np.sum, 0, trace['xi']*trace['beta'])
                            /np.apply_along_axis(np.sum, 0, trace['xi'])
                       })

In [15]:
results.sort_values('inclusion_probability', ascending = False).head(20)

,var,inclusion_probability,beta,beta_given_inclusion
127,127,1.00000,1.080765,1.080765
176,176,0.87925,-0.646303,-0.726684
18,18,0.87875,0.661963,0.753232
241,241,0.78250,0.577308,0.746181
16,16,0.67300,-0.446223,-0.652325
74,74,0.59150,-0.378697,-0.633306
135,135,0.48275,-0.302919,-0.611513
66,66,0.44450,0.286320,0.644222
199,199,0.38375,-0.217808,-0.577155
200,200,0.31225,-0.159157,-0.531241


In [16]:
estimate = trace['beta'] * trace['xi'] 
y_hat = np.apply_along_axis(np.mean, 1, expit(trace['alpha'] + np.dot(X2, np.transpose(estimate) )) )

In [17]:
submission  = pd.DataFrame({'id':df2.id, 'target':y_hat})
submission.to_csv('submission.csv', index = False)

In [18]:
np.mean(y_hat), np.sum(results.inclusion_probability/300)

(0.29129541095602957, 0.052950833333333336)